# 第5讲：相机与图像

视觉SLAM中，观测主要是指**相机成像**的过程。


## 5.1 相机模型

相机将三维世界中的坐标点（米）映射到二维图像平面（像素）的过程能够用几何模型描述，其中最简单模型为**针孔模型**。针孔模型描述了一束光线通过针孔之后，在针孔背面投影成像的关系。但由于相机镜头的透镜作用，光线投影到成像平面的过程中会产生**畸变**。因此，相机投影成像过程需要使用针孔和畸变两个模型来描述，这两个模型能够把外部三维点投影到相机内部成像平面上，构成相机的**内参数**。

### 5.1.1 针孔相机模型

设$O−x−y−z$为相机坐标系，通常$z$轴指向相机前方、$x$向右、$y$向下。$O$为摄像机光心（针孔模型中的针孔）。点$P$经过小孔$O$投影后，落在物理成像平面$O^{\prime}-x^{\prime}-y^{\prime}$上，成像点为$P^{\prime}$。

<img src="./img/fig_5_1.png" width="500" />

在相机坐标系下，设$P$点坐标为$[X, Y, Z]^{\text{T}}$，$P^{\prime}$为$[X^{\prime}, Y^{\prime}, Z^{\prime}]^{\text{T}}$，并且设物理成像平面到小孔的距离为$f$（焦距）。根据三角形相似关系，

$$\frac{Z}{f} = - \frac{X}{X^{\prime}} = - \frac{Y}{Y^{\prime}} \tag {5-1}$$

其中，负号表示成像是倒立的。为简化模型，将成像平面对称到相机前方，在摄像机坐标系中，位于点$P$同侧。此进，可将负号去掉：

$$\frac{Z}{f} = \frac{X}{X^{\prime}} = \frac{Y}{Y^{\prime}} \tag {5-2}$$

整理得：

$$\begin{aligned}
X^{\prime} & = f \frac{X}{Z} \\
Y^{\prime} & = f \frac{Y}{Z} \\
\end{aligned} \tag {5-3}$$


<img src="./img/fig_5_2.png" width="600" />

方程（5-3）表示点$P$与其像之间的空间关系。在相机中，观测数据为像素，因此需要在成像平面上对像进行采样和量化。假设物理成像平面上固定一个像素平面$o − u − v$，在像素平面得到了$P^{\prime}$的像素坐标：$[u, v]^{\text{T}}$。

**像素坐标系（图像坐标系）**的定义为：原点$o^{\prime}$位于图像的左上角，$u$轴向右与$x$轴平行，$v$轴向下与$y$轴平行。像素坐标系与成像平面之间，相差一个缩放和一个原点的平移。假设像素坐标在$u$轴上缩放了$\alpha$倍，在$v$上缩放了$\beta$倍，同时原点平移$[c_{x}, c_{y}]^{\text{T}}$，则$P^{\prime}$的坐标与像素坐标$[u, v]^{\text{T}}$的关系为：

$$\begin{cases}
u = \alpha X^{\prime} + c_{x} \\
v = \alpha Y^{\prime} + c_{y} \\
\end{cases} \tag {5-4}$$

代入方程（5-3）可得

$$\begin{cases}
u = f_{x} \frac{X}{Z} + c_{x} \\
v = f_{y} \frac{Y}{Z} + c_{y} \\
\end{cases} \tag {5-5}$$

其中，$f$的单位为米，$\alpha$、$\beta$单位为像素每米，$f_{x}$、$f_{x}$的单位为像素。使用齐次坐标，将该式写为矩阵形式：

$$\begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \frac{1}{Z} \begin{bmatrix}
    f_{x} & 0 & c_{x} \\
    0 & f_{y} & c_{y} \\
    0 & 0 & 1
\end{bmatrix} \begin{bmatrix}
    X \\ Y \\ Z
\end{bmatrix} \triangleq
\frac{1}{Z} \mathbf{K} \mathbf{P} \tag {5-6}$$

$$Z \begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \begin{bmatrix}
    f_{x} & 0 & c_{x} \\
    0 & f_{y} & c_{y} \\
    0 & 0 & 1
\end{bmatrix} \begin{bmatrix}
    X \\ Y \\ Z
\end{bmatrix} \triangleq
\mathbf{K} \mathbf{P} \tag {5-7}$$

其中，矩阵$\mathbf{K}$称为相机的内参数矩阵（camera intrinsics）。**标定**：确定相机的内参。

相机**外参（camera extrinsics）**指相机的位姿$\mathbf{R}$、$\mathbf{t}$。假设点$P$的世界坐标记为$P_{w}$，其相机坐标是根据相机的位姿，变换到相机坐标系下的结果：

$$Z \mathbf{P}_{uv} = Z \begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \mathbf{K} (\mathbf{R} \mathbf{P}_{w} + \mathbf{t}) = \mathbf{K} \mathbf{T} \mathbf{P}_{w} \tag {5-8}$$

其中，$\mathbf{T}$为欧氏变换矩阵（$\mathbf{R} \mathbf{P}_{w}$中$\mathbf{P}_{w}$为非齐次坐标、$\mathbf{T} \mathbf{P}_{w}$中$\mathbf{P}_{w}$为齐次坐标），方程（5-8）表示点$P$的世界坐标到像素坐标的投影。相机外参随相机的运动而改变，是SLAM中待估计目标，代表着机器人的轨迹。根据齐次坐标性质，方程（5-8）可简化为：

$$\mathbf{P}_{uv} = \mathbf{K} \mathbf{T} \mathbf{P}_{w} \tag {5-9}$$

对于相机坐标系，通常对最后一维归一化处理，得到$P$在相机归一化平面上的投影:

$$\tilde{\mathbf{P}}_{c} = \begin{bmatrix}
    X \\ Y \\ Z
\end{bmatrix} = (\mathbf{T} \mathbf{P}_{w})_{1 : 3},
\mathbf{P}_{c} = \begin{bmatrix}
    X / Z \\ Y / Z \\ 1
\end{bmatrix} \tag {5-10}$$

其中，$\mathbf{P}_{c}$可视为二维齐次坐标，称为归一化坐标，其位于相机前方$z = 1$处的平面上。该平面称为归一化平面。$\mathbf{P}_{c}$经内参后得到像素坐标，因此像素坐标$[u, v]^{\text{T}}$可视为对归一化平面上的点进行量化测量的结果。


### 5.1.2 畸变

透镜会影响成像过程中光线的传播：（1）透镜形状对光线传播的影响：（2）透镜和成像平面不完全平行。

由透镜形状引起的畸变称之为**径向畸变**，透镜使真实环境中直线在图片中变成曲线，越靠近图像的边缘，这种现象越明显。由于透镜一般中心对称，因此这种不规则畸变通常径向对称，主要分为两类：（1）桶形畸变；（2）枕形畸变。

<img src="./img/fig_5_3.png" width="500" />

<img src="./img/fig_5_4.png" width="500" />

平面上，点$P$的极坐标的形式为$[r, \theta]^{\text{T}}$，其中，$\rho$表示点$P$与坐标系原点之间的距离，$\theta$表示与水平轴的夹角。径向畸变表示点$P$沿长度方向的变化$\delta r$，即其距离原点的长度发生变化；切向畸变表示点$P$沿切线方向的变化$\delta \theta$，即水平夹角发生变化。

径向畸变随着与中心的距离增加而增加，可用多项式函数描述畸变导致的坐标变化，这类畸变可用关于与中心间距离的二次（高次）多项式函数纠正：

$$\begin{aligned}
x_{\text{corrected}} = x \left( 1 + k_{1} r^{2} + k_{2} r^{4} + k_{3} r^{6} \right) \\
y_{\text{corrected}} = y \left( 1 + k_{1} r^{2} + k_{2} r^{4} + k_{3} r^{6} \right)
\end{aligned} \tag {5-11}$$

其中，$[x, y]^{\text{T}}$为未纠正的点的坐标，$[x_{\text{corrected}}, y_{\text{corrected}}]^{\text{T}}$为纠正后的点的坐标，注意二者均为归一化平面上的点，而非像素平面上的点。

纠正模型方程（5-11）中，对于畸变较小的图像中心区域，畸变纠正主要由$k_{1}$负责；在畸变较大的边缘区域，畸变纠正主要由$k_{2}$负责。对于普通摄像头，$k_{1}$、$k_{2}$即可纠正径向畸变。而对于畸变很大的摄像头（例如，鱼眼镜头），可以加入$k_{3}$畸变项纠正畸变。

切向畸变的纠正模型为：

$$\begin{aligned}
x_{\text{corrected}} = x + 2 p_{1} x y + p_{2} \left( r^{2} + 2 x^{2} \right) \\
y_{\text{corrected}} = y + p_{1} \left( r^{2} + 2 y^{2} \right) + 2 p_{2} x y \\
\end{aligned} \tag {5-12}$$

因此，合并方程（5-11）、（5-12），对于相机坐标系中点$p(X, Y, Z)$，能够通过五个畸变系数给出点$P$在像素平面上的位置：

1. 将三维空间点（世界坐标系）投影到归一化图像平面（相机坐标系），设其归一化坐标为$[x, y]^{\text{T}}$；

2. 对归一化平面上的点进行径向、切向畸变纠正。

$$\begin{cases}
x_{\text{corrected}} = x \left( 1 + k_{1} r^{2} + k_{2} r^{4} + k_{3} r^{6} \right) + 2 p_{1} x y + p_{2} \left( r^{2} + 2 x^{2} \right) \\
y_{\text{corrected}} = y \left( 1 + k_{1} r^{2} + k_{2} r^{4} + k_{3} r^{6} \right) + p_{1} \left( r^{2} + 2 y^{2} \right) + 2 p_{2} x y \\
\end{cases} \tag {5-13}$$

3. 将纠正后的点通过内参数矩阵投影到像素平面，得到该点在图像上的位置。

$$\begin{cases}
u = f_{x} x_{\text{corrected}} + c_{x} \\
v = f_{y} y_{\text{corrected}} + c_{y} \\
\end{cases} \tag {5-14}$$

畸变处理（undistort，畸变校正）流程：（1）先对整张图像校正畸变，然后考虑畸变校正后图像上点的空间位置；（2）先考虑图像中的某个点，然后根据畸变纠正方程得到该点的空间位置。前者在视觉SLAM中更加常见，因此，当一个图像畸变纠正后，可以直接用针孔模型建立投影关系，而无需考虑畸变。

单目相机成像过程：

1. 给定点$P$，其世界坐标为$\mathbf{P}_{w}$；

2. 给定相机的外参数$\mathbf{R}$、$\mathbf{t}$（$\mathbf{T} \in SE(3)$），点$P$的相机坐标为：

$$\tilde{\mathbf{P}}_{c} = \mathbf{R} \mathbf{P}_{w} + \mathbf{t}$$

3. 将$\tilde{\mathbf{P}}_{c}$投影到归一化平面$z = 1$上，得到点$P$的归一化相机坐标：$\mathbf{P}_{c} = [X / Z, Y / Z, 1]^{\text{T}}$；

4. 点$P$的归一化相机坐标经过内参后，得到其像素坐标：$\mathbf{P}_{uv} = \mathbf{K} \mathbf{P}_{c}$。

### 5.1.3 双目相机模型

由于从相机光心到归一化平面连线上的所有点，都投影至同一个像素，因此，单目相机成像模型无法仅根据一个像素确定该点的在空间中的具体位置。只有当点$P$的深度确定时（例如，双目或RGB-D相机），才能确定其空间位置。

<img src="./img/fig_5_5.png" width="600" />

双目相机原理是根据双目成像物体差异（视差）判断物体与相机的距离：同步采集左右相机图像，计算图像间视差，估计各像素深度。双目相机一般由左、右两个水平放置的相机组成。在左右双目相机中，两个相机都可视作针孔相机，由于水平布置，两个相机的光圈中心都位于$x$轴上，光圈中心间的距离称为双目相机的**基线（baseline）**，记作$b$。

<img src="./img/fig_5_6.png" width="600" />

给定点$P$，其在左、右相机成像分别记作$P_{L}$、$P_{R}$。由于相机基线，$P_{L}$、$P_{R}$的位置存在差异。理想情况下，左、右相机的坐标只在$x$轴上存在差异，则$P$的像也只在$x$轴（图像$u$轴）上存在差异。记点$P$在成像平面上的像素坐标分别为$u_{L}$和$u_{R}$，根据三角形$P − P_{L}− P_{R}$和$P − O_{L}− O_{R}$的相似关系，有：

$$\frac{z - f}{z} = \frac{b - u_{L} + u_{R}}{b} \tag {5-15}$$

整理得：

$$z = \frac{f b}{d}, d = u_{L} - u_{R} \tag {5-16}$$

其中，$d$为点$P$在左、右相机成像的横坐标之差，称为**视差（disparity）**。视差与点$P$到相机的距离成反比：视差越大，距离越近。由于视差最小为一个像素，因此，双目相机的深度存在最大值，由$f b$决定。基线越长，双目相机的最大测量距离越远；反之，小型双目器件只能测量很近的距离。

双目相机估计距离的难点在于*视差$d$计算*。

### 5.1.4 RGB-D相机模型

RGB-D相机能够主动测量每个像素的深度，按实现原理可分为两大类：

1. 通过**红外结构光（structured light）**测量像素距离，如`Kinect`1代、`Project Tango`1代、`Intel RealSense`等；

2. 通过**飞行时间法（time-of-flight, ToF）**原理测量像素距离，如`Kinect`2代和一些ToF传感器等。

RGB-D相机由普通摄像头、发射器、接收器组成，测量距离时，RGB-D相机向探测目标发射一束光线（通常是红外光）：结构光相机根据返回的结构光图案，计算物体离自身的距离。ToF相机向目标发射脉冲光，根据发送到返回之间的光束飞行时间，确定物体离自身的距离。ToF原理和激光传感器十分相似，区别在于激光是逐点扫描获取距离，而ToF相机则可以获得整幅图像的像素深度。测量深度后，RGB-D相机按照生成时各相机的摆放位置，完成深度与彩色图像素之间的配对，输出一一对应的彩色图和深度图。

<img src="./img/fig_5_7.png" width="600" />

根据图像各位置的色彩信息和距离信息，可以计算像素的3D相机坐标，生成点云（point cloud）。RGB-D数据，既可以在图像层面处理，亦可在点云层面处理。RGB-D相机能够实时地测量每个像素点的距离。但其使用范围受限。用红外进行深度值测量的RGB-D相机，容易受到日光或其他传感器发射的红外光干扰，因此不能在室外使用；同时使用多个RGB-D相机也会造成相互干扰。对于透射材质物体，因为接受不到反射光，所以无法测量这些点的位置。

## 5.2 图像

### 5.2.1 计算机中图像的表示

**灰度图**：在灰度图中，每个像素位置$(x, y)$对应一个灰度值$I$，一张宽度$w$、高度$h$的图像，其数学形式可以记成一个矩阵：

$$\mathbf{I} (x, y) \in \R^{w \times h}$$

由于计算机无法表达整个实数空间，所以只能在某个范围内，对图像进行量化。常见灰度图中，用$0 - 255$之间整数（`unsigned char`，字节）表示图像灰度大小。一张分辨率（宽、高）为$640 \times 480$的灰图度可以表示为：

```cpp
unsigned char image[480][640];
```
其中，数组行数（$Y$轴）对应图像高度、列数（$X$轴）对应图像宽度。像素坐标系原点位于图像左上角，$X$轴指向右侧，$Y$轴指向下侧（即$u$、$v$坐标）。

<img src="./img/fig_5_8.png" width="550" />

通过像素坐标，可以实现对像素的读写。例如，对像素$(x, y)$读取：

```cpp
unsigned char pixel = image[y][x];
```

在RGB-D相机的深度图中，记录了各像素离相机的距离，该距离通常以毫米为单位，且RGB-D相机的量程通常为十几米，因此深度图采用十六位整数（`unsigned short`，`0 - 65536`），

彩色图像表示需要引入通道（channel）概念。在计算机中，用红、绿、蓝三种颜色的组合表达任意一种色彩。因此每个像素要记录`RGB`三个数值，每个数值称为一个通道。例如，真彩色图像有三个通道，每个通道都由8位整数表示，一个像素占用24位空间。`OpenCV`彩色图像中，通道默认顺序为`BGR`，即一个24位像素的前8位表示蓝色、中间8位表示绿色、最后8位表示红色。表示图像透明度时，需使用`RGBA`四个通道。


## 5.3 实践：图像的存取与访问

### 5.3.1 安装 OpenCV

1. [下载源码](https://opencv.org/releases/)

依赖项：

```sh
sudo apt-get install build-essential libgtk2.0-dev libvtk5-dev libjpeg-dev libtiff4-dev libjasper-dev libopenexr-dev libtbb-dev
```

2. 编译：

```sh
cmake
make
```

3. 安装
```sh
sudo make install
```

### 5.3.2 操作OpenCV图像

```cpp
// image_basics.cpp

#include <iostream>
#include <chrono>

using namespace std;

#include <opencv2/core/core.hpp>
#include <opencv2/highgui/highgui.hpp>

int main(int argc, char const *argv[])
{
    // 读取argv[1]指定的图像
    cv::Mat image;
    // `cv::imread`函数读取给定路径的图像
    image = cv::imread(argv[1]);

    // 判断图像文件是否正确读取
    if (image.data == nullptr)
    {
        // 数据不存在，可能是文件不存在
        cerr << "文件" << argv[1] << "不存在" << endl;
        return 1;
    }

    // 文件顺利读取，输出基本信息
    cout << "图像宽为" << image.cols << ",高为" << image.rows << ",通道数为" << image.channels() << endl;
    // `cv::imshow`显示图像
    cv::imshow("image", image);
    // 暂停程序，等待按键输入
    cv::waitKey(0);

    // 判断 image 的类型
    if (image.type() != CV_8UC1 && image.type() != CV_8UC3)
    {
        // 图像类型不符合要求
        cout << "请输入一张彩色图或灰度图" << endl;
        return 1;
    }

    // 遍历图像，请注意以下遍历方式亦可使用于随机访问
    // 使用`std::chrono`为算法计时
    chrono::steady_clock::time_point t1 = chrono::steady_clock::now();
    for (size_t y = 0; y < image.rows; y++)
    {
        for (size_t x = 0; x < image.cols; x++)
        {
            // 访问(x, y)处的像素
            // 用`cv::Mat::ptr`获得图像行指针
            // row_ptr是第y行的头指针
            unsigned char * row_ptr = image.ptr<unsigned char>(y);
            // data_ptr指向待访问的像素数据
            unsigned char * data_ptr = &row_ptr[x * image.channels()];
            // 输出该像素的每个通道，如果是灰度图就只有一个通道
            for (int c = 0; c != image.channels(); c++)
            {
                // data为I(x,y)第c个通道的值
                unsigned char data = data_ptr[c];
            }

        }

    }

    chrono::steady_clock::time_point t2 = chrono::steady_clock::now();
    chrono::duration<double> time_used = chrono::duration_cast<chrono::duration<double>>(t2 - t1);
    cout << "遍历图像用时：" << time_used.count() << "秒。" << endl;

    // 关于`cv::Mat`的拷贝
    // 直接赋值并不会拷贝数据
    cv::Mat image_another = image;
    // 修改`image_another`会导致`image`发生变化
    // 将左上角100*100的块置零
    image_another(cv::Rect(0, 0, 100, 100)).setTo(0);
    cv::imshow("image", image);
    cv::waitKey(0);

    // 使用`clone`函数拷贝数据
    cv::Mat image_clone = image.clone();
    image_clone(cv::Rect(0, 0, 100, 100)).setTo(255);
    cv::imshow("image", image);
    cv::imshow("image_clone", image_clone);
    cv::waitKey(0);

    cv::destroyAllWindows();

    return 0;
}

```

```cmake
# CMakeLists.txt

# 声明要求的 cmake 最低版本
cmake_minimum_required( VERSION 2.8 )

# 声明一个 cmake 工程
project( Sophus )

# 添加 c++ 11 标准支持
set( CMAKE_CXX_FLAGS "-std=c++11" )

# OpenCV库
find_package( OpenCV REQUIRED )
# 添加头文件
include_directories( ${OpenCV_INCLUDE_DIRS} )

add_executable( image_basics image_basics.cpp )
# 链接OpenCV库
target_link_libraries( image_basics ${OpenCV_LIBS} )

```

In [5]:
import cv2
import time

def main(filepath):
    
    image = cv2.imread(filepath)
    if image is None:
        print("文件{}不存在".format(filepath))
        return None
        
    print("图像宽为{0[1]}，高为{0[0]}，通道数为{0[2]}".format(image.shape))
    cv2.imshow("image", image)
    cv2.waitKey(0)
    
    t1 = time.time()
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            for c in range(image.shape[2]):
                data = image[y, x, c]
                
    t2 = time.time()
    print("遍历图像用时：{}秒。".format(t2 - t1) )
    
    image_another = image
    image_another[0 : 100, 0 : 100, :] = 0
    cv2.imshow("image", image)
    cv2.waitKey(0)
    
    image_clone = image.copy()
    image_clone[0 : 100, 0 : 100, :] = 255
    cv2.imshow("image", image)
    cv2.imshow("image_clone", image_clone)
    cv2.waitKey(0)
    
    cv2.destroyAllWindows()
    
    return None

if __name__ == "__main__":
    
    filepath = "./img/fig_2_7.png"
    main(filepath)
    

图像宽为890，高为341，通道数为3
遍历图像用时：0.22542595863342285秒。


## 5.4 实践：拼接点云

位姿格式（`pose.txt`）：平移向量、旋转四元数

$$[x, y, z, q_{x}, q_{y}, q_{z}, q_{w}]$$

其中，$q_{w}$为四元数实部。

任务：

1. 根据内参计算RGB-D图像对应的点云；

2. 根据相机位姿（外参），将点云拼接，组成地图。

点云库：`PCL（point cloud library）`

```sh
sudo add-apt-repository ppa:v-launchpad-jochen-sprickerhof-de/pcl
sudo apt-get update
# ubuntu 14
sudo apt-get install libpcl-all
# ubuntu 16 +
sudo apt-get install libpcl-dev
```